# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import google_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# The path to our CSV file
csv_path = "cities.csv"
city_df = pd.read_csv(csv_path)

city_df.head()

,city,cloudiness,country,date,humidity,lat,Lng,Temp,Wind
0,Yellowknife,5,CA,1579350882,73,62.46,-114.35,-34.01,4.70
1,Mpongwe,100,ZM,1579350884,90,-13.51,28.16,74.88,5.91
2,Barrow,1,US,1579350885,77,71.29,-156.79,5.00,3.36
3,Bluff,96,NZ,1579350886,84,-46.60,168.33,63.00,8.01
4,Bossangoa,27,CF,1579350887,11,6.49,17.46,97.97,7.56


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Store latitude and longitude in locations
locations = city_df[["lat", "Lng"]]

# Fill NaN values and convert to float
humidity = city_df["humidity"]

In [6]:
 # Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [9]:
ideal_city_df = city_df.query('Temp < 80 & Temp >= 70 & Wind < 10 & cloudiness == 0')
ideal_city_df

,city,cloudiness,country,date,humidity,lat,Lng,Temp,Wind
41,Tura,0,IN,1579350928,57,25.52,90.22,70.05,1.92
149,Basudebpur,0,IN,1579351045,39,21.14,86.74,76.44,8.95
413,Partapur,0,IN,1579351337,30,23.60,74.17,73.89,6.44
506,Goure,0,NE,1579351438,23,13.98,10.27,77.00,8.99


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [14]:
hotel_df = ideal_city_df
hotel_df["Hotel Name"] = ""
hotel_df["Hotel Address"] = ""
hotel_df["Hotel Rating"] = ""
hotel_df

/opt/anaconda3/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/anaconda3/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/anaconda3/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFram

,city,cloudiness,country,date,humidity,lat,Lng,Temp,Wind,Hotel Name,Hotel Address,Hotel Rating
41,Tura,0,IN,1579350928,57,25.52,90.22,70.05,1.92,,,
149,Basudebpur,0,IN,1579351045,39,21.14,86.74,76.44,8.95,,,
413,Partapur,0,IN,1579351337,30,23.60,74.17,73.89,6.44,,,
506,Goure,0,NE,1579351438,23,13.98,10.27,77.00,8.99,,,


In [18]:
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotel",
    "key": google_key
}

# Use the lat/lng we recovered to identify airports
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
   
    #print the name_address url, avoid doing for public github repos in order to avoid exposing key
    #print(name_address.url)

    # convert to json
    name_address = name_address.json()
    #print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
        hotel_df.loc[index, "Hotel Address"] = name_address["results"][0]["vicinity"]
        hotel_df.loc[index, "Hotel Rating"] = name_address["results"][0]["rating"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

Missing field/result... skipping.


In [17]:
hotel_df


,city,cloudiness,country,date,humidity,lat,Lng,Temp,Wind,Hotel Name,Hotel Address,Hotel Rating
41,Tura,0,IN,1579350928,57,25.52,90.22,70.05,1.92,Hotel Polo Orchid,"Araimile, Tura",3.6
149,Basudebpur,0,IN,1579351045,39,21.14,86.74,76.44,8.95,Sneha Residency,"Arandua - Basudevpur Rd, Basudebpur",3.9
413,Partapur,0,IN,1579351337,30,23.60,74.17,73.89,6.44,Hotel Royal Inn,"Tehsil Road, NH927A, Adinath colony, Partapur",4
506,Goure,0,NE,1579351438,23,13.98,10.27,77.00,8.99,,,


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display Map